In [84]:
import numpy as np
import matplotlib.pyplot as plt
import graphviz

## Построить граф

In [139]:
def BuildGraph(data):
    graph = {}
    
    for line in data:
        tokens = line.strip().split(' ')
        node = tokens[0]
        graph[node] = []
        
        for i in range(1, len(tokens)):
            succ = tokens[i]
            graph[node].append(succ)
            
            if succ not in graph:
                graph[succ] = []

    return graph

## Проверить граф на ацикличность

In [140]:
def IsGraphAcycle(graph):
    is_asycle = True
    colors = {}

    for key in graph.keys():
        colors[key] = 'white'
    
    def IsGraphAcycleRec(node):
        colors[node] = 'grey'

        for succ in graph[node]:
            if (colors[succ] == 'white'):
                if IsGraphAcycleRec(succ) == False:
                    return False
                    
            if (colors[succ] == 'grey'):
                return False
        
        colors[node] = 'black'
        return True

    for node in graph.keys():
        is_asycle = is_asycle and IsGraphAcycleRec(node)
        
    return is_asycle

## Построить Start ноду

In [141]:
def FindRoots(graph):
    roots = set()

    # Найти корень данного подграфа
    def FindRootRec(current):
        is_intermediate = False
        
        for [node, succs] in graph.items():
            if current in succs:
                FindRootRec(node)
                is_intermediate = True

        if not is_intermediate:
            roots.add(current)

    for node in graph.keys():
        FindRootRec(node)

    return list(roots)


def AddStartNode(graph, name):
    graph[name] = FindRoots(graph)

## Построить End ноду

In [142]:
def FindLeaves(graph):
    leaves = []
    
    for [node, succs] in graph.items():
        if len(succs) == 0:
            leaves.append(node)

    return leaves


def AddEndNode(graph, name):
    leaves = FindLeaves(graph)
    graph[name] = []
    
    for leaf in leaves:
        graph[leaf] = [name]

## Топологически пронумеровать граф

In [143]:
def BuildTopologicalNumbering(graph, start):
    colors = {}
    rev_stack = []
    numbering = {}
    
    for node in graph.keys():
        colors[node] = 'white'

    def BuildTopologicalNumberingRec(node):
        if colors[node] == 'black':
            return

        if colors[node] == 'grey':
            assert 0, 'Topological numbering cannot be used for cycled graph!'

        if colors[node] == 'white':
            colors[node] = 'grey'

            for succ in graph[node]:
                BuildTopologicalNumberingRec(succ)

            colors[node] = 'black'
            rev_stack.insert(0, node)


    BuildTopologicalNumberingRec(start)

    for i in range(len(rev_stack)):
        node = rev_stack[i]
        numbering[node] = i

    return numbering

## Перевернуть граф

In [144]:
def BuildFlippedGraph(graph, start):
    preds = {}

    for node in graph.keys():
        preds[node] = set()

    def BuildFlippedGraphRec(node):
        for succ in graph[node]:
            preds[succ].add(node)
            BuildFlippedGraphRec(succ)

    BuildFlippedGraphRec(start)

    for node in preds:
        preds[node] = list(preds[node])
    
    return preds

## Построить дерево доминаторов

In [145]:
def BuildDomTree(graph, start):
    flipped_graph = BuildFlippedGraph(graph, start)
    numbering = BuildTopologicalNumbering(graph, start)
    doms = {}

    for node in graph.keys():
        doms[node] = set()
        doms[node].add(node)

    # Спускаемся от Start вниз
    for i in range(len(graph.keys())):
        # Найти ноду с номером i
        node = list(numbering.keys())[list(numbering.values()).index(i)]
        tmp = set()
        preds = flipped_graph[node]

        # Пересекаем множества dom для всех preds
        if len(preds) != 0:
            tmp.update(doms[preds[0]])
            
            for j in range(1, len(preds)):
                tmp.intersection_update(doms[preds[j]])

        # Объединяем текущую ноду и полученное пересечение
        doms[node].update(tmp)

    idoms = {}

    for node in doms.keys():
        indexes = []
        
        for dom in doms[node]:
            indexes.append(numbering[dom])

        indexes.sort()
        
        if len(indexes) == 1:
            # Один dom => нет Idom
            idoms[node] = None
        else:
            # Препдоследний номер - номер Idom
            idom_index = indexes[-2]

            # Поиск Idom по номеру
            idom = list(numbering.keys())[list(numbering.values()).index(idom_index)]
            idoms[node] = idom

    idom_tree = {}

    # Построить дерево доминаторов из словаря {node: idom} 
    for idom in idoms.keys():
        idom_tree[idom] = []

        for node in idoms.keys():
            if idoms[node] == idom:
                idom_tree[idom].append(node)
    
    return idom_tree

## Отрисовать граф

In [146]:
def RenderGraph(graph, name, xlabel = None):
    dot = graphviz.Digraph(name, format='png')
    
    for [node, succs] in graph.items():
        dot.node(node, node)

        if xlabel:
            dot.body.append(f'\t{node} [xlabel = {xlabel[node]}]\n')
        
        for succ in succs:
            dot.edge(node, succ)
    
    dot.render()

# Работа с конкретным графом

In [148]:
data = []
filename = 'data.txt'

with open(filename, "r") as lines:
    for line in lines:
        data.append(line)

# Построить граф
graph = BuildGraph(data)

# Проверить на ацикличность
print("Is graph acycle:", IsGraphAcycle(graph))

# Добавить Start и End ноды
AddStartNode(graph, 'Start')
AddEndNode(graph, 'End')

# Построить перевернутый граф
flipped_graph = BuildFlippedGraph(graph, 'Start')

# Пронумеровать оба графа
numbering = BuildTopologicalNumbering(graph, 'Start')
flipped_numbering = BuildTopologicalNumbering(flipped_graph, 'End')

# Построить DomTree и PostDomTree
dom_tree = BuildDomTree(graph, 'Start')
post_dom_tree = BuildDomTree(flipped_graph, 'End')

# Вывести графы
RenderGraph(graph, 'Graph', numbering)
RenderGraph(flipped_graph, 'FlippedGraph', flipped_numbering)
RenderGraph(dom_tree, 'DomTree')
RenderGraph(post_dom_tree, 'PostDomTree')

Is graph acycle: True
